# **TDA on MNIST digit images**

Contents replicated from giotto-tda example: [MNIST_classification](https://github.com/giotto-ai/giotto-tda/blob/master/examples/MNIST_classification.ipynb)

In [ ]:
!pip install tensorflow
!pip install keras
!pip install giotto-tda

In [18]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import fashion_mnist

# Load MNIST dataset
(x_mnist, y_mnist), (x_mnist_test, y_mnist_test) = mnist.load_data()

# Load Fashion-MNIST dataset
(x_fashion, y_fashion), (x_fashion_test, y_fashion_test) = fashion_mnist.load_data()

print(f"X MNIST shape: {x_mnist_test.shape}, y MNIST shape: {y_mnist_test.shape}")
print(f"X Fashion shape: {x_fashion_test.shape}, y Fashion shape: {y_fashion_test.shape}")

X MNIST shape: (10000, 28, 28), y MNIST shape: (10000,)
X Fashion shape: (10000, 28, 28), y Fashion shape: (10000,)


# From pixels to topological features

Since our images are made of pixels, it is convenient to use filtrations of cubical complexes instead of simplicial ones.

## Greyscale image

In [19]:
import numpy as np
from gtda.plotting import plot_heatmap

# Show the first 8
im8_idx = np.flatnonzero(y_mnist_test == 8)[0]
img8 = x_mnist_test[im8_idx] # Shape (28,28)
plot_heatmap(img8)

## Binarized image

Filtrations of cubical complexes are built from binary images consisting of only black and white pixels. We can convert our greyscale image to binary by applying a threshold on each pixel value via the Binarizer transformer:

In [20]:
from gtda.images import Binarizer

# Reshape to (n_samples, n_pixels_x, n_pixels_y) format
im8 = x_mnist_test[im8_idx][None,:,:]

binarizer = Binarizer(threshold=0.4)
im8_binarized = binarizer.fit_transform(im8)

binarizer.plot(im8_binarized)

## Filtered image

We can use a wide range of filtrations. Full list here: https://giotto-ai.github.io/gtda-docs/latest/modules/images.html#filtrations
This example will use radial filtration, which assigns to each pixel a value corresponding to its distance from a predefined center of the image

In [21]:

from gtda.images import RadialFiltration

# We pick center = (20,6)
radial_filtration = RadialFiltration(center=np.array([20, 6]))
im8_filtration = radial_filtration.fit_transform(im8_binarized)

radial_filtration.plot(im8_filtration, colorscale="jet")

Here, the pixel values increase as we move from the upper-right to bottom-left of the image. These pixel values can be used to define a filtration of cubical complexes, where K_i contains all pixels with value less than the ith smallest pixel value in the greyscale image

## Persistence diagram

In this case, we use CubicalPersistence transformer which is the cubical analogue to simplicial transformers like VietorisRipsPersistence

In [22]:
from gtda.homology import CubicalPersistence

cubical_persistence = CubicalPersistence(n_jobs=-1)
im8_cubical = cubical_persistence.fit_transform(im8_filtration)

cubical_persistence.plot(im8_cubical)

We can see three H_1 generators corresponding to the loops in the 8 digit.

As a postprocessing step, it is convenient to rescale the persistence diagrams

In [23]:
from gtda.diagrams import Scaler

scaler = Scaler()
im8_scaled = scaler.fit_transform(im8_cubical)

scaler.plot(im8_scaled)

## Vectorial representation

The final step is to define a vectorial representation of the persistence diagram that can be used to obtain machine learning features. Following our example from Figure 2, we convolve our persistence diagram with a Gaussian kernel and symmetrize along the main diagonal, a procedure achieved via the HeatKernel transformer:

In [24]:
from gtda.diagrams import HeatKernel

heat = HeatKernel(sigma=.15, n_bins=60, n_jobs=-1)
im8_heat = heat.fit_transform(im8_scaled)

# Visualise the heat kernel for H1
heat.plot(im8_heat, homology_dimension_idx=1, colorscale='jet')

## Pipeline

In [25]:
from sklearn.pipeline import Pipeline
from gtda.diagrams import Amplitude

steps = [
    ("binarizer", Binarizer(threshold=0.4)),
    ("filtration", RadialFiltration(center=np.array([20, 6]))),
    ("diagram", CubicalPersistence()),
    ("rescaling", Scaler()),
    ("amplitude", Amplitude(metric="heat", metric_params={'sigma':0.15, 'n_bins':60}))
]

heat_pipeline = Pipeline(steps)

In the final step we've used the Amplitude transformer to "vectorize" the persistence diagram via the heat kernel method above. In our example, this produces a vector of amplitudes a = (a_0, a_1) where each amplitude
a_i corresponds to a given homology dimension in the persistence diagram. By extracting these feature vectors from each image, we can feed them into a machine learning classifier

In [26]:
im8_pipeline = heat_pipeline.fit_transform(im8)
im8_pipeline

array([[1.40667732e-04, 4.10738182e+00]])

# Feature extraction pipeline

We augment our radial filtration with a height filtration, defined by choosing a unit vector v in some *direction* and assigning values based on the distance of a pixel to the hyperplane defined by v.

We also generate features from persistence diagrams using persistence entropy.

In [27]:
from sklearn.pipeline import make_pipeline, make_union
from gtda.diagrams import PersistenceEntropy
from gtda.images import HeightFiltration

direction_list = [[1, 0], [1, 1], [0, 1], [-1, 1], [-1, 0], [-1, -1], [0, -1], [1, -1]]

center_list = [
    [13, 6],
    [6, 13],
    [13, 13],
    [20, 13],
    [13, 20],
    [6, 6],
    [6, 20],
    [20, 6],
    [20, 20],
]

# Creating a list of all filtration transformer, we will be applying
filtration_list = (
    [
        HeightFiltration(direction=np.array(direction), n_jobs=-1)
        for direction in direction_list
    ]
    + [RadialFiltration(center=np.array(center), n_jobs=-1) for center in center_list]
)

# Creating the diagram generation pipeline
diagram_steps = [
    [
        Binarizer(threshold=0.4, n_jobs=-1),
        filtration,
        CubicalPersistence(n_jobs=-1),
        Scaler(n_jobs=-1),
    ]
    for filtration in filtration_list
]

# Listing all metrics we want to use to extract diagram amplitudes
metric_list = [
    {"metric": "bottleneck", "metric_params": {}},
    {"metric": "wasserstein", "metric_params": {"p": 1}},
    {"metric": "wasserstein", "metric_params": {"p": 2}},
    {"metric": "landscape", "metric_params": {"p": 1, "n_layers": 1, "n_bins": 100}},
    {"metric": "landscape", "metric_params": {"p": 1, "n_layers": 2, "n_bins": 100}},
    {"metric": "landscape", "metric_params": {"p": 2, "n_layers": 1, "n_bins": 100}},
    {"metric": "landscape", "metric_params": {"p": 2, "n_layers": 2, "n_bins": 100}},
    {"metric": "betti", "metric_params": {"p": 1, "n_bins": 100}},
    {"metric": "betti", "metric_params": {"p": 2, "n_bins": 100}},
    {"metric": "heat", "metric_params": {"p": 1, "sigma": 1.6, "n_bins": 100}},
    {"metric": "heat", "metric_params": {"p": 1, "sigma": 3.2, "n_bins": 100}},
    {"metric": "heat", "metric_params": {"p": 2, "sigma": 1.6, "n_bins": 100}},
    {"metric": "heat", "metric_params": {"p": 2, "sigma": 3.2, "n_bins": 100}},
]

#
feature_union = make_union(
    *[PersistenceEntropy(nan_fill_value=-1)]
    + [Amplitude(**metric, n_jobs=-1) for metric in metric_list]
)

tda_union = make_union(
    *[make_pipeline(*diagram_step, feature_union) for diagram_step in diagram_steps],
    n_jobs=-1
)

In [28]:
from sklearn import set_config
set_config(display='diagram')

tda_union

FeatureUnion(n_jobs=-1,
             transformer_list=[('pipeline-1',
                                Pipeline(steps=[('binarizer',
                                                 Binarizer(n_jobs=-1,
                                                           threshold=0.4)),
                                                ('heightfiltration',
                                                 HeightFiltration(direction=array([1, 0]),
                                                                  n_jobs=-1)),
                                                ('cubicalpersistence',
                                                 CubicalPersistence(n_jobs=-1)),
                                                ('scaler', Scaler(n_jobs=-1)),
                                                ('featureunion',
                                                 FeatureUnion(transformer_list=[('persistenceentropy',
                                                                                 Persist...
                                                                                           metric_params={'n_bins': 100,
                                                                                                          'p': 1,
                                                                                                          'sigma': 1.6},
                                                                                           n_jobs=-1)),
                                                                                ('amplitude-11',
                                                                                 Amplitude(metric='heat',
                                                                                           metric_params={'n_bins': 100,
                                                                                                          'p': 1,
                                                                                                          'sigma': 3.2},
                                                                                           n_jobs=-1)),
                                                                                ('amplitude-12',
                                                                                 Amplitude(metric='heat',
                                                                                           metric_params={'n_bins': 100,
                                                                                                          'p': 2,
                                                                                                          'sigma': 1.6},
                                                                                           n_jobs=-1)),
                                                                                ('amplitude-13',
                                                                                 Amplitude(metric='heat',
                                                                                           metric_params={'n_bins': 100,
                                                                                                          'p': 2,
                                                                                                          'sigma': 3.2},
                                                                                           n_jobs=-1))]))]))])

# Amplitude & Entropy Test

In [30]:
from sklearn.model_selection import train_test_split

train_size, test_size = 60, 10

# Reshape to (n_samples, n_pixels_x, n_pixels_y)
X = x_mnist_test.reshape((-1, 28, 28))
y = y_mnist_test

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=train_size, test_size=test_size, stratify=y, random_state=666
)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


X_train_tda = tda_union.fit_transform(X_train)
X_train_tda.shape

X_train shape: (60, 28, 28), y_train shape: (60,)
X_test shape: (10, 28, 28), y_test shape: (10,)


(60, 476)

In [38]:
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import Amplitude, PersistenceEntropy
from keras.datasets import mnist

# Load MNIST data
(x_mnist, y_mnist), (x_mnist_test, y_mnist_test) = mnist.load_data()

# Filter images of digits 4 and 8
images_4 = x_mnist_test[y_mnist_test == 4][:50]
images_8 = x_mnist_test[y_mnist_test == 8][:50]

# Transform each image to a point cloud
def img_to_point_cloud(img):
    return [(i, j, img[i][j]) for i in range(img.shape[0]) for j in range(img.shape[1])]

point_clouds_4 = np.array([img_to_point_cloud(img) for img in images_4])
point_clouds_8 = np.array([img_to_point_cloud(img) for img in images_8])

# Compute persistent homology
VR = VietorisRipsPersistence(metric='euclidean', max_edge_length=255., homology_dimensions=[0, 1])

diagrams_4 = VR.fit_transform(point_clouds_4)
diagrams_8 = VR.fit_transform(point_clouds_8)


In [39]:
# Compute amplitude
amplitude = Amplitude(metric='bottleneck')
print("Amplitude 4:", np.mean(amplitude.fit_transform(diagrams_4)))
print("Amplitude 8:", np.mean(amplitude.fit_transform(diagrams_8)))

# Compute entropy
entropy = PersistenceEntropy(normalize=True)
print("Entropy 4:", np.mean(entropy.fit_transform(diagrams_4)))
print("Entropy 8:", np.mean(entropy.fit_transform(diagrams_8)))

Amplitude 4: 6.151146583557129
Amplitude 8: 6.1476606595516206
Entropy 4: 1.0032744002077438
Entropy 8: 0.991891619407439


In [40]:
images_8_2 = x_mnist_test[y_mnist_test == 8][50:100]
point_clouds_8_2 = np.array([img_to_point_cloud(img) for img in images_8_2])

diagrams_8_2 = VR.fit_transform(point_clouds_8_2)
# Compute amplitude of two sets of 8's
amplitude = Amplitude(metric='bottleneck')
print("Amplitude 1st set:", np.mean(amplitude.fit_transform(diagrams_8)))
print("Amplitude 2nd set:", np.mean(amplitude.fit_transform(diagrams_8_2)))

# Compute entropy
entropy = PersistenceEntropy(normalize=True)
print("Entropy 1st set:", np.mean(entropy.fit_transform(diagrams_8)))
print("Entropy 2nd set:", np.mean(entropy.fit_transform(diagrams_8_2)))

Amplitude 1st set: 6.1476606595516206
Amplitude 2nd set: 6.0096497386693954
Entropy 1st set: 0.991891619407439
Entropy 2nd set: 0.9899752899028542
